# 线性方程组求解
> 书第五章计算实习题1（P78）

In [437]:
using LinearAlgebra,BenchmarkTools,LambdaFn

include("MatrixDisp.jl")
include("Swap.jl")

eye(n) = Matrix{Float64}(I,n,n)

eye (generic function with 1 method)

In [438]:
@latex A = [10 -7 0 1;-3 2.099999 6 2;5 -1 5 -1;2 1 0 2]

<center>$A=\begin{bmatrix}10.0&-7.0&0.0&1.0\\-3.0&2.099999&6.0&2.0\\5.0&-1.0&5.0&-1.0\\2.0&1.0&0.0&2.0\end{bmatrix}$</center>


In [439]:
@latex b = [8;5.900001;5;1]

<center>$b=\begin{bmatrix}8.0\\5.900001\\5.0\\1.0\end{bmatrix}$</center>


## LU分解方法计算

In [170]:
function LU!(A::Matrix)
    n = size(A)[1]
    @assert n == size(A)[2] "The Matrix must be square matrix!"
    L = eye(n)
    U = zeros(n,n)
    for r = 1:n
        U[r:r,r:n] = A[r:r,r:n] - L[r:r,1:r-1]*U[1:r-1,r:n]
        if r < n
            L[r+1:n,r:r] = (A[r+1:n,r:r] - L[r+1:n,1:r-1]*U[1:r-1,r:r])/U[r,r]
        end
    end
    return L,U
end

LU! (generic function with 1 method)

In [440]:
L,U=LU!(A);
@latex L

<center>$L=\begin{bmatrix}1.0&0.0&0.0&0.0\\-0.3&1.0&0.0&0.0\\0.5&-2.499999999650555e6&1.0&0.0\\0.2&-2.3999999996645334e6&0.9599996800001067&1.0\end{bmatrix}$</center>


In [441]:
@latex U

<center>$U=\begin{bmatrix}10.0&-7.0&0.0&1.0\\0.0&-1.000000000139778e-6&6.0&2.3\\0.0&0.0&1.5000004997903332e7&5.749998499196276e6\\0.0&0.0&0.0&5.079998907178727\end{bmatrix}$</center>


In [442]:
@latex δA = L*U - A

<center>$δA=\begin{bmatrix}0.0&0.0&0.0&0.0\\0.0&0.0&0.0&-2.220446049250313e-16\\0.0&1.1102230246251565e-16&0.0&0.0\\0.0&4.440892098500626e-16&6.4116548942624e-10&0.0\end{bmatrix}$</center>


In [114]:
DetA = U |> diag |> prod

-762.0000900767544

In [162]:
function LinearSolveWithLU!(L::Matrix,U::Matrix,b::Vector)
    n = size(b)[1]
    y = zeros(n)
    for i ∈ 1:n
        y[i:i,1:1] = b[i:i,1:1] - L[i:i,1:i-1]*y[1:i-1,1:1]
    end
    x = zeros(n)
    for i ∈ n:-1:1
        x[i:i,1:1] = (y[i:i,1:1]-U[i:i,i+1:n]*x[i+1:n,1])/U[i,i]
    end
    return x
end

LinearSolveWithLU! (generic function with 1 method)

In [443]:
@latex x=LinearSolveWithLU!(L,U,b)

<center>$x=\begin{bmatrix}-6.09103523174781e-10\\-1.0000000008881784\\1.0000000000483817\\0.9999999998737863\end{bmatrix}$</center>


## 列主元的LU分解

In [349]:
function P⁻¹LU!(A::Matrix)
    # deepcopy otherwise you will change the origin data
    A = A |> deepcopy
    n = size(A)[1]
    @assert n == size(A)[2] "The Matrix must be square matrix!"
    Iₚ = zeros(Int8,n)
    for r = 1:n
        #1 calculate s
        if r != 1
            A[r:n,r:r] -= A[r:n,1:r-1]*A[1:r-1,r:r]
        end
        
        #2 choose main element
        iᵣ = r-1+argmax(abs.(A[r:n,r]))
        Iₚ[r] = iᵣ
        
        #3 swap row of A
        @swap A[r:r,:],A[iᵣ:iᵣ,:]
        
        #4 calculate L,U same as function LU!
        A[r+1:n,r:r] /= A[r,r]
        if 1 < r < n
            A[r:r,r+1:n] -= A[r:r,1:r-1]*A[1:r-1,r+1:n]
        end
    end
    return A,Iₚ
end

P⁻¹LU! (generic function with 1 method)

In [444]:
LUᵤₙᵢₒₙ,Iₚ=P⁻¹LU!(A);
@latex LUᵤₙᵢₒₙ

<center>$LUᵤₙᵢₒₙ=\begin{bmatrix}10.0&-7.0&0.0&1.0\\0.5&2.5&5.0&-1.5\\-0.3&-4.000000000559112e-7&6.000002&2.2999994\\0.2&0.9600000000000002&-0.7999997333334223&5.079998906667031\end{bmatrix}$</center>


In [445]:
@latex Iₚ

<center>$Iₚ=\begin{bmatrix}1\\3\\3\\4\end{bmatrix}$</center>


In [352]:
function LinearSolveWithP⁻¹LU!(LUᵤₙᵢₒₙ ::Matrix,Iₚ::Vector,b::Vector)
    b = b |> copy
    n = length(Iₚ)
    for i = 1:n-1
        if i != Iₚ[i]
            @swap b[i],b[Iₚ[i]]
        end
    end
    for i = 2:n
        b[i] -= LUᵤₙᵢₒₙ[i,1:i-1]'*b[1:i-1]
    end
    b[n] /= LUᵤₙᵢₒₙ[n,n]
    for i = n-1:-1:1
        b[i] -= LUᵤₙᵢₒₙ[i,i+1:n]'*b[i+1:n]
        b[i] /= LUᵤₙᵢₒₙ[i,i]
    end
    return b
end

LinearSolveWithP⁻¹LU! (generic function with 1 method)

In [446]:
@latex x = LinearSolveWithP⁻¹LU!(LUᵤₙᵢₒₙ,Iₚ,b)

<center>$x=\begin{bmatrix}2.6645352591003756e-16\\-0.9999999999999997\\0.9999999999999999\\1.0\end{bmatrix}$</center>
